# Regressão

A tarefa de **regressão** consiste em predizer **valores** para determinados **objetos**. Vamos contextualizar de maneira operacional esses objetos e valores através de seus tipos de variáveis: em geral um valor é um número real <code>(float)</code> ou um inteiro <code>(int)</code>; já o objeto a ser regredido pode ser um **vetor de features** - aqui vetor tem o mesmo sentido de uma variável lista <code>(lst)</code> ou uma tupla <code>(tuple)</code>. 

Diferentes **algoritmos de regressão** podem ser usados para realizar essa tarefa de, dada uma entrada, gerar uma saída (valor predito para aquele objeto).

Define-se a regressão como uma tarefa de **aprendizado de máquina supervisionado**, isto é, os algoritmos de regressão precisam ser treinados com pares objeto-valor tidos como associações corretas. Somente após o treinamento é que o algoritmo de regressão estará apto a predizer valores para novas entradas (das quais não se sabe o valor a princípio) - diz-se que  algoritmo aprendeu o padrão nos dados de treinamento e agora pode ser usado para predizr o valor de saída para novas entradas.

-----------------------------------------------

In [ ]:
# No google colab é preciso atualizar a versão do matplotlib para gerar alguns detalhes nos gráficos
#!pip install matplotlib --upgrade 

## 1 - IArpi Data Set

#### Descrição geral:
Três diferentes objetos são postos a se mover em um plano inclinado devido a ação da gravidade. Atributos cinemáticos do movimento dos corpos são coletados. Pretende-se estabelecer uma relação entre esses atributos e cada tipo de objeto.

#### Objetivo:
O problema consiste na predição da velocidade média de três objetos (esfera, cilindro e aro) a partir da sua altura inicial em um plano inclinado a um determinado ângulo. O objetivo é introduzir técnicas de IA para cursos de graduação de física onde o experimento do plano inclinado é amplamente estudado (de maneira teórica e em laboratório).


#### Features (variáveis de entrada):
As features foram determinadas experimentalmente:
- Ângulo: ângulo de inclinação do plano [°]
- Altura: de partida do objeto [m]
- Tipo de objeto (esfera, cilindro, aro)

#### Alvo (valor de saída):
- Velocidade Média: velocidade média determinada pela distância/tempo [m/s]

#### Referências:
- https://github.com/simcomat/IArpi
- Ferreira, H., Almeida Junior, E. F., Espinosa-García, W., Novais, E., Rodrigues, J. N. B., & Dalpian, G. M. (2022). Introduzindo aprendizado de máquina em cursos de física: o caso do rolamento no plano inclinado. In Revista Brasileira de Ensino de Física (Vol. 44). FapUNIFESP (SciELO). https://doi.org/10.1590/1806-9126-rbef-2022-0214 

In [ ]:
import pandas as pd # Para trabalhar com dados na forma de tabela
import numpy as np  # Para trabalhar com vetores e matrizes

from sklearn.model_selection import train_test_split   # Separação treino/teste
from sklearn.preprocessing import MinMaxScaler         # Escalonador

# Algoritmos de Regressão
from sklearn.linear_model import LinearRegression      # Regressao Linear
from sklearn.svm import SVR                            # Regressão por Máquina de Vetor Suporte
from sklearn.tree import DecisionTreeRegressor         # Regressão por Árvore de Decisão
from sklearn.neighbors import KNeighborsRegressor      # k-vizinhos mais próximos (KNN)
from sklearn.ensemble import RandomForestRegressor     # RandomForest
from sklearn.ensemble import GradientBoostingRegressor # GradientBoosting
from sklearn.neural_network import MLPRegressor        # Multilayer Perceptron

# Validação cruzada
from sklearn.model_selection import KFold            # Para separar os dados em k folds na regressao
from sklearn.model_selection import cross_validate   # Para rodar treinamento e teste sobre kfolds
from sklearn import preprocessing          # Auxilia na transformação dos dados (passo 3)
from sklearn.pipeline import make_pipeline # Permite realizar uma sequência de processamentos

# Métricas de desempenho
from sklearn.metrics import mean_absolute_error, r2_score       # Métricas de Regressão
from sklearn.metrics import make_scorer


# Módulos para plotar gráficos e ajustar formatação dos mesmos
import matplotlib.pyplot as plt
import matplotlib.transforms as mtransforms
from matplotlib.ticker import AutoMinorLocator
from matplotlib.lines import Line2D
import seaborn as sns

# Para não aparecer avisos de warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Parâmetros gerais para os gráficos
# Definição dos tamanhos de fontes e ticks dos gráficos
fsize = 12
tsize = 10
major = 5.0
minor = 3.0

style = 'default'
plt.style.use(style)

#plt.rcParams['text.usetex'] = True  # Para usar fonte tex (precisa instalar o tex antes)
plt.rcParams['font.size'] = fsize      
plt.rcParams['legend.fontsize'] = tsize
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor

### 1 - Primeiro passo: carregando os dados

In [ ]:
tabela_dados = pd.read_csv('https://raw.githubusercontent.com/simcomat/IArpi/main/datasets/rolling.csv', sep=';') 

In [ ]:
tabela_dados.head()

In [ ]:
tabela_dados.info()

### 1 - Segundo passo: separação de dados

Os algoritmos de regressão não estão preparados para receber um nome (string) como entrada. Então, apenas fornecer o nome do objeto (esfera, cilindro, aro) para o modelo não irá funcionar. Precisamos primeiro converter esses nomes em uma representação numérica. Para isso vamos usar o **OneHot Encoding**. Nessa representação, cada valor do atributo objeto torna-se uma característica única (feature). Para 3 valores de objetos teremos 3 novas colunas com valores binários (verdadeiro ou falso, 1 ou 0) representando cada objeto, por exemplo, se seguirmos (esfera, cilindro, aro) uma esfera será representada pela tupla (1,0,0) enquanto um aro será por (0,0,1). Fisicamente, misturas não seriam possiveis como uma esfera-aro (1,0,1), mas em outras situações essa técnica pode ser usada para representar a presença de mais um objeto (como duas palavras diferentes em uma mesma frase).

In [ ]:
ohe = pd.get_dummies(tabela_dados.Objeto, prefix='objeto') # One hot encoding
tabela_dados = tabela_dados.join(ohe)

In [ ]:
tabela_dados.head()

In [ ]:
x = tabela_dados[['Altura (m)','Ângulo (°)', 'objeto_aro', 'objeto_cilindro','objeto_esfera']] 
y = tabela_dados['Velocidade Média (m/s)'] # Atributo alvo

# Dividindo conjunto de treinamento e conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

### 1 - Terceiro passo: transformação dos dados

**OBS:** perceba que nem sempre a transformação ocorre apenas no terceiro passo. Tivemos que fazer uma transformação usando o One Hot Encoding (OHE) no segundo passo. A troca da ordem neste caso foi feita para facilitar a etapa de separação (não ter que filtrar a coluna objeto após transformar via OHE.

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### 1 - Quarto passo: treinando os algoritmos de regressão

In [ ]:
# Regressao Linear
lr = LinearRegression()
lr.fit(x_train_scaled,y_train)

# KNN Regressor
knnr = KNeighborsRegressor()
knnr.fit(x_train_scaled,y_train)

# SVM
svmr = SVR()
svmr.fit(x_train_scaled,y_train)

# Regressão por Árvore de Decisão
dtr = DecisionTreeRegressor()
dtr.fit(x_train_scaled,y_train)

# Regressão por Random
rfr = RandomForestRegressor(random_state=42)
rfr.fit(x_train_scaled, y_train)

# Regressõ por GB
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(x_train_scaled, y_train)

# Multilayer Perceptron
mlpr =  MLPRegressor(random_state=42)
mlpr.fit(x_train_scaled,y_train)

regressores = {
    'LR':lr,
    'KNNR':knnr,
    'SVMR':svmr,
    'RFR':rfr,
    'GBR':gbr,
    'MLPR':mlpr,
}

### 1 - Quinto passo: avaliar o desempenho

In [ ]:
resultados={}
resultados_MAE={}
resultados_R2={}
resultados_MADMAE={}
mad = y_test.mad()

for rg_name, rg in regressores.items():
    
    y_pred = rg.predict(x_test_scaled)        # Entrando os dados de teste no modelo ML
    mae = mean_absolute_error(y_test, y_pred) # Calculando métrica MAE
    r2 = r2_score(y_test, y_pred)             # Calculando métrica R2
    
    # Salvando resultados
    scoring = {'MAE': mae,
               'R2' :r2,
               'MAD:MAE':mad/mae
         }
    resultados[rg_name]=scoring
    resultados_MAE[rg_name]=mae
    resultados_R2[rg_name]=r2
    resultados_MADMAE[rg_name]=mad/mae

In [ ]:
resultado_teste_regressao = pd.DataFrame(data=resultados)
resultado_teste_regressao.head()

### 1 - Extra: Calculando o modelo físico

Iremos fazer o cálculo pontual do valor de $v_{med}$ para cada dado experimental, seguindo:

### $v_{med}=\frac{1}{2}\sqrt{\frac{2gh}{1+\beta}}$


Vamos definir uma função para isso:

In [ ]:
def encontra_velocidade(altura, objeto0, objeto1, objeto2):
    g=9.8 
    # O MF sabe o beta devido a teoria
    if objeto0==1:
        beta=2/5 # Esfera
    elif objeto1==1:
        beta=1/2 # Cilindro
    elif objeto2==1:
        beta=1 # Aro
    vel_med = (0.5)*(2*g*altura/(1+beta))**0.5
    return vel_med

In [ ]:
x_test2 = x_test.copy() 
x_test2['Vel MF'] = x_test2.apply(lambda x: encontra_velocidade(x['Altura (m)'], x['objeto_esfera'],
                                                            x['objeto_cilindro'], x['objeto_aro']), axis=1)
y_pred_modelo = np.array(x_test2['Vel MF'])

resultados_MAE['MF'] = mean_absolute_error(y_test, y_pred_modelo)
resultados_R2['MF'] = r2_score(y_test, y_pred_modelo)
resultados_MADMAE['MF'] = mad/resultados_MAE['MF']
resultados['MF'] = {'MAE': resultados_MAE['MF'],
                    'R2' :resultados_R2['MF'],
                    'MAD:MAE' :resultados_MADMAE['MF']
                    }

In [ ]:
resultado_teste_regressao = pd.DataFrame(data=resultados)
resultado_teste_regressao.head()

Comparando os desempenhos dos modelos:

In [ ]:
y_pred= knnr.predict(x_test_scaled)

In [ ]:
# Figura Regressor
fig, axd = plt.subplot_mosaic([['(a)', '(c)', '(d)'],
                               ['(b)', '(c)', '(d)']],
                              figsize=(11, 4.5), constrained_layout=True)


# Gráficos de barras (a) e (b)
clrs = ['#81BC82' for i in regressores]+['#d5e6ac'] # Definição de cores em código hexadecimal
sns.barplot(x=list(resultados_MAE.keys()), y =[round(resultados_MAE[k],3) for k in resultados_MAE.keys()],
            ax=axd['(a)'], palette=clrs, edgecolor='grey')
sns.barplot(x=list(resultados_R2.keys()),  y =[round(resultados_R2[k],3) for k in resultados_R2.keys()],
            ax=axd['(b)'], palette=clrs, edgecolor='grey')

axd['(b)'].set_xlabel('Algoritmos de Regressão')
axd['(a)'].set_ylabel('MAE (m/s)'), axd['(a)'].set_ylim([0,0.06])
axd['(b)'].set_ylabel('$R^2$'), axd['(b)'].set_ylim([0,1])
axd['(a)'].set_yticks([0, 0.04, 0.08])
axd['(b)'].set_yticks([0, 0.5, 1])
axd['(b)'].bar_label(axd['(b)'].containers[0],  rotation=90, label_type='center', color='white')
axd['(a)'].set_title('Métricas')
plt.setp(axd['(a)'].get_xticklabels(), visible=False)
axd['(b)'].tick_params(axis='x', rotation=45)

pos=0.05
colors_mae=['w','k','w','k','k','w','w']  # Cores dos valores expressos nas barras do item (a)
y_pos_mae=[0.027,0.038,0.033,0.038,0.038,0.027,0.04] # Altura dos valores expressos nas abrras do item (a)
for i, kp in enumerate([round(resultados_MAE[k],3) for k in resultados_MAE.keys()]):       
    axd['(a)'].text(pos,y_pos_mae[i],
             s =  '{0:.3f}'.format(kp),
             color=colors_mae[i],
             rotation=90,
             horizontalalignment='center',
             verticalalignment='top',
             multialignment='center')
    pos = pos+1


# Grafico de dispersão ML
sns.scatterplot(x=y_test,y=y_pred, label='$y_{pred}$', color='#81BC82', edgecolor='k',
                marker='o', s=25, ax=axd['(c)'])
axd['(c)'].plot([0,1], [0,1], color='r', linestyle='dashed', linewidth = 1,
                  label='$y_{pred}=y_{test}$') # Reta 100% correto
axd['(c)'].set(xlabel='Velocidade Média \n Experimental (m/s)', ylabel='Velocidade Média Predita (m/s)')
axd['(c)'].legend(loc=4)
axd['(c)'].set_xlim([0.2,0.8]), axd['(c)'].set_ylim([0.2,0.8])
axd['(c)'].set_xticks([0.2, 0.4, 0.6, 0.8]), axd['(c)'].set_yticks([0.2, 0.4, 0.6, 0.8])
axd['(c)'].xaxis.set_minor_locator(AutoMinorLocator())
axd['(c)'].yaxis.set_minor_locator(AutoMinorLocator())
axd['(c)'].grid(alpha=0.2, linestyle='-.', color='k', linewidth =1)
axd['(c)'].set_title('Aprendizado de Máquina')

# Grafico de dispersão MF
sns.scatterplot(x=y_test,y=y_pred_modelo, label='$y_{pred}$', color='#d5e6ac', edgecolor='k',
                marker='o', s=25, ax=axd['(d)'])
axd['(d)'].plot([0,1], [0,1], color='r', linestyle='dashed', linewidth = 1,
                  label='$y_{pred}=y_{test}$') # Reta 100% correto
axd['(d)'].set(xlabel='Velocidade Média \n Experimental (m/s)', ylabel='Velocidade Média Predita (m/s)')
axd['(d)'].legend(loc=4)
axd['(d)'].set_xlim([0.2,0.8]), axd['(d)'].set_ylim([0.2,0.8])
axd['(d)'].set_xticks([0.2, 0.4, 0.6, 0.8]), axd['(d)'].set_yticks([0.2, 0.4, 0.6, 0.8])
axd['(d)'].xaxis.set_minor_locator(AutoMinorLocator())
axd['(d)'].yaxis.set_minor_locator(AutoMinorLocator())
axd['(d)'].grid(alpha=0.2, linestyle='-.', color='k', linewidth =1)
axd['(d)'].set_title('Modelo Físico')
axd['(d)'].yaxis.tick_right()
axd['(d)'].set_ylabel('')

# Escrevendo os itens (a), (b), ... em cada um dos gráficos da figura
for label, ax in axd.items():
    trans = mtransforms.ScaledTranslation(10/72, -5/72, fig.dpi_scale_trans)
    ax.text(0.0, 1.0, label, transform=ax.transAxes + trans,
            fontsize='medium', verticalalignment='top')
    
#plt.savefig("regressao.pdf", format="pdf") # Para salvar a imagem em formato pdf

### 1 - Extra: Validação cruzada

Vamos usar a validação cruzada para verificar se o desempenho de cada modelo muda com a partição de treino e teste usada.

In [ ]:
x = tabela_dados[['Altura (m)','Ângulo (°)', 'objeto_aro', 'objeto_cilindro','objeto_esfera']] 
y = tabela_dados['Velocidade Média (m/s)'] # Atributo alvo

In [ ]:
lr = LinearRegression()                          # Regressao Linear
knnr = KNeighborsRegressor()                     # KNN Regressor
svmr = SVR()                                     # SVM
dtr = DecisionTreeRegressor()                    # Regressão por Árvore de Decisão
rfr = RandomForestRegressor(random_state=42)     # Regressão por Random
gbr = GradientBoostingRegressor(random_state=42) # Regressão por GB
mlpr =  MLPRegressor(random_state=42)            # Multilayer Perceptron

regressores = {
    'LR':lr,
    'KNNR':knnr,
    'SVMR':svmr,
    'RFR':rfr,
    'GBR':gbr,
    'MLPR':mlpr,
    'MF':'encontra_velocidade'
}

In [ ]:
# Separando 5 folds garantimos usar 20% dos dados para teste e 80% para treinamento
kf = KFold(n_splits=5, shuffle=True, random_state=42)    

results = []
for clf_name, clf in regressores.items():                       # Para cada modelo testado
    
    r2_list=[]
    mae_list=[]
    
    for i, (train_index, test_index) in enumerate(kf.split(x)): # Para cada fold de dados
        
        # Separando conjunto de treino e teste
        x_train = x.iloc[train_index]
        y_train = y.iloc[train_index]
        x_test = x.iloc[test_index]
        y_test = y.iloc[test_index]
        
        # Modelo fisico
        if clf_name == 'MF':
            y_pred = x_test.apply(lambda x: encontra_velocidade(x['Altura (m)'], x['objeto_esfera'],
                                                            x['objeto_cilindro'], x['objeto_aro']), axis=1)
            
            # Avaliando
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            
            # Salvando as métricas do fold
            mae_list.append(mae)
            r2_list.append(r2)
            
        # Modelos de Machine Learning
        else:
            
            # Escalonando
            scaler = MinMaxScaler()
            scaler.fit(x_train)
            x_train_scaled = scaler.transform(x_train)
            x_test_scaled = scaler.transform(x_test)

            # Treinando e predizendo
            clf.fit(x_train_scaled,y_train)
            y_pred = clf.predict(x_test_scaled)
            
            # Avaliando
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            
            # Salvando as métricas do fold
            mae_list.append(mae)
            r2_list.append(r2)
                
    ob = {
        'regressor':clf_name,
        'mae_avg':np.mean(mae_list),
        'mae_std':np.std(mae_list),
        'r2_avg':np.mean(r2_list),
        'r2_std':np.std(r2_list)
    }
    results.append(ob)


df_reg = pd.DataFrame(data= results)

In [ ]:
df_reg.head(10)

In [ ]:
# Figura Validação Cruzada Classificação e Regressão
fig, axd = plt.subplot_mosaic([['(a)',],
                               ['(b)']],
                              figsize=(4, 4), constrained_layout=True)

# Gráficos de barras (c) e (d)
clrs = ['#81BC82' for i in regressores]+['#d5e6ac'] # Definição de cores em código hexadecimal
sns.barplot(x='regressor', y ='mae_avg', data=df_reg,
            ax=axd['(a)'], palette=clrs, edgecolor='grey')
sns.barplot(x='regressor',  y ='r2_avg', data=df_reg,
            ax=axd['(b)'], palette=clrs, edgecolor='grey')

axd['(a)'].set_xlabel(''),
axd['(b)'].set_xlabel('Algoritmos de Regressão')
axd['(a)'].set_ylabel('MAE (m/s)'), axd['(a)'].set_ylim([0,0.1])
axd['(b)'].set_ylabel('$R^2$'), axd['(b)'].set_ylim([0,1])
axd['(a)'].set_yticks([0, 0.05,  0.1])
axd['(b)'].set_yticks([0, 0.5, 1])
axd['(a)'].bar_label(axd['(a)'].containers[0],  rotation=90, label_type='edge', color='k', fmt='%.2f')
axd['(b)'].bar_label(axd['(b)'].containers[0],  rotation=90, label_type='center', color='white', fmt='%.2f')


plt.setp(axd['(a)'].get_xticklabels(), visible=False)
axd['(b)'].tick_params(axis='x', rotation=45)


# Barras verticais indicando variabilidade pelo desvio padraõ
x_coords = [p.get_x() + 0.5 * p.get_width() for p in axd['(a)'].patches]
y_coords = [p.get_height() for p in axd['(a)'].patches]
axd['(a)'].errorbar(x=x_coords, y=y_coords, yerr=df_reg['mae_std'], fmt="none", c="r", capsize=0.1)

x_coords = [p.get_x() + 0.5 * p.get_width() for p in axd['(b)'].patches]
y_coords = [p.get_height() for p in axd['(b)'].patches]
axd['(b)'].errorbar(x=x_coords, y=y_coords, yerr=df_reg['r2_std'], fmt="none", c="r", capsize=0.1)


#plt.savefig("regressao.pdf", format="pdf") # Para salvar a imagem em formato pdf

### 1 - Extra: regressão de equação predefinida

Vamos definir uma função e então usá-la para regredir os dados expeirmentais.

A princípio podemos definir qualquer função:

In [ ]:
def raiz(x, a, b):
    return np.sqrt(a*x)+b

In [ ]:
raiz(4, 1, 0) # Estamos passando os coeficientes

Para entrontrar os coeficientes que melhore ajustam a curva a uma série de dados, podems utilizar a biblioteca SicPy. Logo, o que estamos de fato fazendo é uma regressão onde a curva assumida não é mais a equação linear $\hat{y}=ax+b$ e sim uma expressão genérica que definimos anteriormente.

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# Assumindo a queda livre com um ruido gaussiano nos dados de velocidade média coletados
# loc=media, scale=desvio padrao

x_exemplo = np.linspace(0, 4, 50)
y_exemplo = raiz(x_exemplo, 2*10/4, 0) +  np.random.normal(loc=0, scale=0.2, size=x_exemplo.size)

Regredindo a função definida em raiz com os dados:

In [ ]:
popt, pcov = curve_fit(raiz, x_exemplo, y_exemplo)

In [ ]:
popt  # Coeficientes estimados e pcov  e a matriz de covariancia

In [ ]:
y_modelo = raiz(x_exemplo, popt[0], popt[1]) # Calculando o modelo encontrado

In [ ]:
plt.plot(x_exemplo, y_exemplo, 'b*')
plt.plot(x_exemplo, y_modelo, 'r-')
plt.xlim([0,4]),plt.ylim([0,5])
plt.xlabel('Altura [m]'), plt.ylabel('Velocidade média [m/s]')
plt.show()

No caso do dados reais do plano inclinado, temos um fator complicador: para cada objeto os coeficientes esperados para a função devem ser diferentes. Então, apesar de estarmos "mais corretos" no modelo matemático assumido *a priori*, devemos "adicionar mais física" ao problema, selecionando cada parcela de dado que será inserida no modelo:

In [ ]:
# Selecionando os objetos
dados_aro = tabela_dados[tabela_dados['objeto_aro']==1]
dados_esfera = tabela_dados[tabela_dados['objeto_esfera']==1]
dados_cilindro = tabela_dados[tabela_dados['objeto_cilindro']==1]

# Separando treino e teste
x = tabela_dados[['Altura (m)','Ângulo (°)', 'objeto_aro', 'objeto_cilindro','objeto_esfera']] 
y = tabela_dados['Velocidade Média (m/s)'] # Atributo alvo

# Regredindo os dados
popt_aro, pcov = curve_fit(raiz, dados_aro['Altura (m)'], dados_aro['Velocidade Média (m/s)']) 
popt_esfera, pcov = curve_fit(raiz, dados_esfera['Altura (m)'], dados_esfera['Velocidade Média (m/s)']) 
popt_cilindro, pcov = curve_fit(raiz, dados_cilindro['Altura (m)'], dados_cilindro['Velocidade Média (m/s)'])

# Calcuando os modelos regredidos
y_aro = raiz(dados_aro['Altura (m)'], popt_aro[0], popt_aro[1])
y_esfera = raiz(dados_esfera['Altura (m)'], popt_esfera[0], popt_esfera[1])
y_cilindro = raiz(dados_cilindro['Altura (m)'], popt_cilindro[0], popt_cilindro[1])

In [ ]:
fig, axs = plt.subplots(1,3, figsize=(8,3))

sns.scatterplot(data = dados_aro, x ='Altura (m)', y='Velocidade Média (m/s)',  ax = axs[0], color='b')
sns.lineplot(x = dados_aro['Altura (m)'], y = y_aro,  ax = axs[0], color='b')

sns.scatterplot(data = dados_esfera, x ='Altura (m)', y='Velocidade Média (m/s)',  ax = axs[1], color='r')
sns.lineplot(x = dados_esfera['Altura (m)'], y=y_esfera,  ax = axs[1], color='r')

sns.scatterplot(data = dados_cilindro, x ='Altura (m)', y='Velocidade Média (m/s)',  ax = axs[2], color='g')
sns.lineplot(x = dados_cilindro['Altura (m)'], y=y_cilindro,  ax = axs[2], color='g')

fig.tight_layout()

In [ ]:
print(popt_aro, popt_esfera, popt_cilindro)

In [ ]:
def calcula_beta(coef_a):
    g = 9.8
    return (g/(2*coef_a))-1

In [ ]:
print('Beta aro: ', calcula_beta(popt_aro[0]))            # Esperado  B= 1
print('Beta esfera: ', calcula_beta(popt_esfera[0]))      # Esperado  B= 2/5 = 0.4
print('Beta cilindro: ', calcula_beta(popt_cilindro[0]))  # Esperado  B= 1/2 = 0.5

### 1 - Extra: Regressão Simbólica

Os diferentes métodos de regressão nada mais são do que diferentes estratégias para encontrar coeficientes de funções pré-definidas (métodos paramétricos como regressão linear e SVM) ou para otimizar uma função custo sem conexão direta com o problema (métodos não paramétricos como KNN e Árvore de Decisão).

Agora poderíamos estar interessados em um problema mais geral: supor uma expressão que relaciona as entradas em saída, e não apenas os coeficiente que melhor se adequam a uma presuposição original.

Neste caso, o "modelo aprendido" será exatamente a expressão matemática que melhor relaciona os dados.

A tarefa de se regredir expressões matemáticas dá-se o nome de Regressão Simbólica. Vejamos o exemplo sobre todos os dados:


In [ ]:
#!pip install gplearn

In [ ]:
from gplearn.genetic import SymbolicRegressor

In [ ]:
# Instanciando o algorimto de regressao simbolica
# Algoritmos Genéticos são usados para reslver esse problema
model = SymbolicRegressor(population_size=5000, generations=20, p_crossover=0.7,
                          function_set=('add', 'sub', 'mul', 'div', 'sqrt'),
                          p_subtree_mutation=0.1, p_hoist_mutation=0.05, p_point_mutation=0.1,
                          max_samples=0.9, parsimony_coefficient=0.01, stopping_criteria=0.01,
                          verbose=1, random_state=0)

In [ ]:
model.fit( np.array(dados_aro['Altura (m)']).reshape(-1,1), dados_aro['Velocidade Média (m/s)'])

In [ ]:
model.fit( np.array(dados_esfera['Altura (m)']).reshape(-1,1), dados_esfera['Velocidade Média (m/s)'])

In [ ]:
model.fit( np.array(dados_cilindro['Altura (m)']).reshape(-1,1), dados_cilindro['Velocidade Média (m/s)'])

A expressão encontrada (sqrt) é exatamente aquela que melhor descreve o fenômeno de acordo com os pressupostos físicos.

Podemos comparar os coeficientes encontrados pela Regressão Simbólica e pela regressão com função arbitrária $\sqrt{a\bf{x}}+b$

In [ ]:
rs_aro = 0.311
rs_esfera = 0.226
rs_cilindro = 0.251

print(1/rs_aro, 1/rs_esfera, 1/rs_cilindro) # Método de regressão simbólica
print(popt_aro[0], popt_esfera[0], popt_cilindro[0]) # Método de regressão pre-definida

-----------------------------

## 2 - MeltingPoint Dataset

#### Descrição geral:
São fornecidos dados de compostos orgânicos e seus respectivos pontos de fusão (temperatura de transição entre forma sólida e líquida).

#### Objetivo:
Prever a temperatura de fusão baseado nas cacaterísticas observadas.

#### Features (variáveis de entrada):
São fornecidos 202 descritores das moléculas em 2D e 3D.

#### Alvo (valor de saída):
- MTP: temperatura de fusão. 

#### Referências:
- M. Karthikeyan, Robert C. Glen e Andreas Bender, General Melting Point Prediction Based on a Diverse Compound Data Set and Artificial Neural Networks, https://pubs.acs.org/doi/10.1021/ci0500132.
- D. Krstajic, L. J Buturovic, D. E. Leahy, e S. Thomas, Cross-validation pitfalls when selecting and assessing regression and classification models, https://jcheminf.biomedcentral.com/articles/10.1186/1758-2946-6-10.

In [ ]:
# Bibliotecas utilizadas

import pandas as pd  # Biblioteca para trabalhar com dados na forma de tabela
import numpy as np   # Trabalhar com matrizes e vetores
import time          # Para medir tempo

# Bibliotecas para gerar gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Aprendizado de Máquina
from sklearn.model_selection import train_test_split   # Separação treino/teste
from sklearn.preprocessing import MinMaxScaler         # Escalonador

# Algoritmos de Regressão
from sklearn.linear_model import LinearRegression      # Regressão Linear
from sklearn.linear_model import Ridge                 # Regressão Ridge
from sklearn.cross_decomposition import PLSRegression  # Partial-least Squares (PLS)
from sklearn.svm import SVR                            # Regressão por Máquina de Vetor Suporte (SVM)
from sklearn.tree import DecisionTreeRegressor         # Regressão por Árvore de Decisão
from sklearn.neighbors import KNeighborsRegressor      # k-vizinhos mais próximos (KNN)
from sklearn.ensemble import RandomForestRegressor     # RandomForest (RF)
from sklearn.ensemble import GradientBoostingRegressor # GradientBoosting (GB)
from sklearn.neural_network import MLPRegressor        # Multilayer Perceptron

# Validação cruzada
from sklearn.model_selection import KFold            # Para separar os dados em k folds na regressao
from sklearn.model_selection import cross_validate   # Para rodar treinamento e teste sobre kfolds
from sklearn import preprocessing          # Auxilia na transformação dos dados (passo 3)
from sklearn.pipeline import make_pipeline # Permite realizar uma sequência de processamentos

# Métricas de desempenho
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score       # Métricas de Regressão
from sklearn.metrics import make_scorer

### 2 - Primeiro passo: carregar os dados e entendê-los

Os dados brutos estão no arquivo <code>4137_185_92_DataSetMTPSMIDescr.txt</code>:

In [ ]:
#!wget https://pubs.acs.org/doi/suppl/10.1021/ci0500132/suppl_file/ci0500132si20050112_060506.zip

In [ ]:
#!unzip ci0500132si20050112_060506.zip

In [ ]:
df_mp = pd.read_csv('4137_185_92_DataSetMTPSMIDescr.txt', sep='\t')

In [ ]:
df_mp.head(100)

In [ ]:
df_mp.info(verbose=True, null_counts=True)

Agora vamos renomear a coluna <code>\$Field_2</code> para <code>MTP</code> e a coluna <code>\$Field_1</code> para <code>Molecule</code> para facilitar nossa identificação.

A coluna <code>SMILE_Molecule</code> tem a representação SMILES da molécula considerada (uma representação visual da molécula pode ser obtida em https://cdb.ics.uci.edu/cgibin/Smi2DepictWeb.py).

A coluna <code>MTP</code> é a temperatura de fusão em °C.

In [ ]:
df_mp.rename(columns={'$Field_2': 'MTP'}, inplace=True)
df_mp.rename(columns={'$Field_1': 'SMILE_Molecule'}, inplace=True)

Temos um total de 4450 materiais. Entretanto, não temos todos os dados (valores das colunas) para todos eles. Então, vamos deletar as linhas que possuem pelo menos um valor vazio:

In [ ]:
df_mp2 = df_mp.dropna() # Por padrão o dropna() deleta linhas vazias

In [ ]:
len(df_mp), len(df_mp2) # Perdemos cerca de 50 materiais para garantir que todos teram todas as características

Agora vamos ver se existem dados duplicados. Consideraremos duplicados apenas aqueles que tem SMILES iguais.

In [ ]:
df_mp3 = df_mp2.drop_duplicates(subset=['SMILE_Molecule'], keep=False)
len(df_mp), len(df_mp2), len(df_mp3)

Pelo artigo original dos dados, a coluna <code>Case</code> representa o grupo de dados, sendo <code>Case=0</code> o grupo de 4173 estruturas exatraídas do Molecular Diversity Preservation International (MDPI) database e <code>Case=1</code> referem-se a 277 fármacos extraídos do Merck Index e compilados por Bergstrom et al. 

Vamos ver as contagens reais desses agrupamentos agora que deletamos uma parte dos materiais considerados. 

In [ ]:
df_mp3.groupby('Case').count()['SMILE_Molecule']

In [ ]:
# Distribuições da temperatura de fusão
fig, axs = plt.subplots(1,2, figsize=(7,3))
sns.histplot(ax=axs[0], data=df_mp3, hue='Case', x='MTP', kde=True)
sns.boxplot(ax=axs[1], data=df_mp3, x='Case', y='MTP')
fig.subplots_adjust(wspace=0.3, hspace=0.3)

De vez usar o <code>Case=1</code> como <code>test_set</code> como foi feito no artigo original de Karthikeyan et al., ou considerar apenas o dataset do <code>Case=0</code> tanto como <code>train_set</code> quanto como <code>test_set</code> conforme realizado por Krstajic et al., iremos assumir que ambos os casos são representativos de um mesmo fenômeno (a temperatura de fusão de moléculas orgânicas). Considerando tudo como um único dataset, iremos dividir os conjuntos de treino e teste de maneira aleatória, seguindo metodologia de 80% para dados de treino e 20% para dados de teste:

### 2 - Segundo passo: separar os dados

Temos originalmente 202 descritores (atributos descritivos ou <code>features</code>), considerando que <code>MTP</code> é nossa variável alvo, <code>SMILE_Molecule</code> é uma string que não pode ser interpretada pelo nosso modelo e <code>Case</code> é uma variável com um único valor. 

In [ ]:
x = df_mp3.drop(columns=['Case','MTP', 'SMILE_Molecule']) # Estamos tirando a coluna de grupo e a coluna alvo 
y = df_mp3['MTP'] # Atributo alvo

# Dividindo conjunto de treinamento e conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

### 2 - Terceiro passo: transformação dos dados

Vamos escalonar as entradas para que fiquem entre 0 e 1:

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### 2 - Quarto passo: treinando os algoritmos de regressão

In [ ]:
# Regressao Linear
start_time = time.time()
lr = LinearRegression()
lr.fit(x_train_scaled, y_train)
print("LinearReg Tempo de treinamento: {:.3f} segundos".format(time.time() - start_time))

# Ridge Regression
start_time = time.time()
rdg = Ridge()
rdg.fit(x_train_scaled, y_train)
print("Ridge Tempo de treinamento: {:.3f} segundos".format(time.time() - start_time))


# Partial least Squares PLS
start_time = time.time()
pls = PLSRegression()
pls.fit(x_train_scaled, y_train)
print("PLS Tempo de treinamento: {:.3f} segundos".format(time.time() - start_time))

# KNN Regressor
start_time = time.time()
knnr = KNeighborsRegressor()
knnr.fit(x_train_scaled,y_train)
print("KNN Tempo de treinamento: {:.3f} segundos".format(time.time() - start_time))

# SVM
start_time = time.time()
svmr = SVR()
svmr.fit(x_train_scaled,y_train)
print("SVM Tempo de treinamento: {:.3f} segundos".format(time.time() - start_time))

# Regressão por Árvore de Decisão
start_time = time.time()
dtr = DecisionTreeRegressor()
dtr.fit(x_train_scaled,y_train)
print("DT Tempo de treinamento: {:.3f} segundos".format(time.time() - start_time))

# Regressão por RandomForest
start_time = time.time()
rfr = RandomForestRegressor(random_state=42)
rfr.fit(x_train_scaled, y_train)
print("RF Tempo de treinamento: {:.3f} segundos".format(time.time() - start_time))

# Regressõ por GB
start_time = time.time()
gbr = GradientBoostingRegressor(random_state=42)
gbr.fit(x_train_scaled, y_train)
print("GB Tempo de treinamento: {:.3f} segundos".format(time.time() - start_time))

# Multilayer Perceptron
start_time = time.time()
mlpr =  MLPRegressor(random_state=42)
mlpr.fit(x_train_scaled,y_train)
print("MLP Tempo de treinamento: {:.3f} segundos".format(time.time() - start_time))

In [ ]:
regressores = {
    'LR':lr,
    'RR':rdg,
    'PLS':pls,
    'KNNR':knnr,
    'SVMR':svmr,
    'DT':dtr,
    'RFR':rfr,
    'GBR':gbr,
    'MLPR':mlpr,
}

### 2 - Quinto passo: avaliar o desempenho

In [ ]:
resultados={}
mad_MTP = y_test.mad()

for rg_name, rg in regressores.items():
    
    y_pred = rg.predict(x_test_scaled)        # Entrando os dados de teste no modelo ML
    mae = mean_absolute_error(y_test, y_pred) # Calculando métrica MAE
    r2 = r2_score(y_test, y_pred)             # Calculando métrica R2
    
    # Salvando resultados
    scoring = {'MAE': mae,
               'R2' :r2,
               'MAD:MAE':(mad_MTP)/mae
         }
    resultados[rg_name]=scoring

In [ ]:
resultado_teste_regressao = pd.DataFrame(data=resultados)
resultado_teste_regressao.head()

### 2 - Extra: Validação Cruzada

Para verificar a variabilidade inerente a escolha dos dados (entre treinamento e teste), vamos realizar uma validação cruzada:

In [ ]:
x = df_mp3.drop(columns=['Case','MTP', 'SMILE_Molecule']) # Estamos tirando a coluna de grupo e a coluna alvo 
y = df_mp3['MTP'] # Atributo alvo

Modelos já foram criados nas células acima. Basta realizarmos a validação cruzada:

In [ ]:
start_time = time.time() # Marcar o tempo de início

# Separando 5 folds garantimos usar 20% dos dados para teste e 80% para treinamento
# Parâmtro shuffl embaralha os dados antes de dividir os folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)    

results = []
for clf_name, clf in regressores.items():                       # Para cada modelo testado
    
    r2_list=[]
    mae_list=[]
    mad_list=[]
    
    for i, (train_index, test_index) in enumerate(kf.split(x)): # Para cada fold de dados
        
        # Separando conjunto de treino e teste
        x_train = x.iloc[train_index]
        y_train = y.iloc[train_index]
        x_test = x.iloc[test_index]
        y_test = y.iloc[test_index]
        
        # Escalonando
        scaler = MinMaxScaler()
        scaler.fit(x_train)
        x_train_scaled = scaler.transform(x_train)
        x_test_scaled = scaler.transform(x_test)
        
        # Treinando e predizendo
        clf.fit(x_train_scaled,y_train)
        y_pred = clf.predict(x_test_scaled)

        # Avaliando
        mad = y_test.mad() # Variabilidade intrinseca dos dados de saída
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        # Salvando as métricas do fold
        mae_list.append(mae)
        r2_list.append(r2)
        mad_list.append(mad)
     
    ob = {
        'regressor':clf_name,
        'mae_avg':np.mean(mae_list),
        'mae_std':np.std(mae_list),
        'r2_avg':np.mean(r2_list),
        'r2_std':np.std(r2_list),
        'mad_avg:mae_avg':np.mean(mad_list)/np.mean(mae_list),
    }
    results.append(ob)

print("Tempo de execução da validação cruzada: {:.3f} segundos".format(time.time() - start_time))
df_reg = pd.DataFrame(data= results)

In [ ]:
df_reg.head(10)

In [ ]:
df_reg = df_reg.drop(index=0) # Vamos excluir a regressão linear do gráfico final

In [ ]:
df_reg

In [ ]:
# Figura Validação Cruzada Classificação e Regressão
fig, axd = plt.subplot_mosaic([['(a)',],
                               ['(b)']],
                              figsize=(4, 4), constrained_layout=True)

# Gráficos de barras (a) e (b)
sns.barplot(x='regressor', y ='mae_avg', data=df_reg,
            ax=axd['(a)'], color='#81BC82', edgecolor='grey')
sns.barplot(x='regressor',  y ='r2_avg', data=df_reg,
            ax=axd['(b)'], color='#81BC82', edgecolor='grey')

axd['(a)'].set_xlabel(''),
axd['(b)'].set_xlabel('Algoritmos de Regressão')
axd['(a)'].set_ylabel('MAE (°C)'), axd['(a)'].set_ylim([0,0.1])
axd['(b)'].set_ylabel('$R^2$'), axd['(b)'].set_ylim([0,1])
axd['(a)'].set_yticks([0, 30,  60])
axd['(b)'].set_yticks([0, 0.5, 1])
axd['(a)'].bar_label(axd['(a)'].containers[0],  rotation=90, label_type='center', color='white', fmt='%.2f')
axd['(b)'].bar_label(axd['(b)'].containers[0],  rotation=90, label_type='center', color='white', fmt='%.2f')


plt.setp(axd['(a)'].get_xticklabels(), visible=False)
axd['(b)'].tick_params(axis='x', rotation=45)


# Barras verticais indicando variabilidade pelo desvio padraõ
x_coords = [p.get_x() + 0.5 * p.get_width() for p in axd['(a)'].patches]
y_coords = [p.get_height() for p in axd['(a)'].patches]
axd['(a)'].errorbar(x=x_coords, y=y_coords, yerr=df_reg['mae_std'], fmt="none", c="r", capsize=0.1)

x_coords = [p.get_x() + 0.5 * p.get_width() for p in axd['(b)'].patches]
y_coords = [p.get_height() for p in axd['(b)'].patches]
axd['(b)'].errorbar(x=x_coords, y=y_coords, yerr=df_reg['r2_std'], fmt="none", c="r", capsize=0.1)


#plt.savefig("regressao.pdf", format="pdf") # Para salvar a imagem em formato pdf

### 2 - Extra: Retornando para o segundo passo, Seleção de Features

Os algoritmos treinados demosntraram capacidade preditiva, entretanto com um baixo desemepnho (gostariamos de R²>0.8 e MAD:MAE>5).

Vamos retornar ao segundo passo de **seleção de features** e aplicar alguns métodos para diminuir a quantidade de features usadas. Apesar disso não necessariamente aumentar o desempenho de predição, permite uma **redução de dimensionalidade** dos dados de entrada do algoritmo, o que deve diminuir o tempo de treinamento dos algoritmos.

Existem vários métodos de **seleção de features**, como métdos de **filtro**, métodos **Wrapper** e métodos **embutidos**. 

Os métodos de **filtro** envolvem a seleção prévia das features através de algum critério. Esse critério pode ser de **domínio** (por exemplo, um físico escolhendo os atributos que são relevantes para o fenômeno baseado em seu conhecimento da física do fenômeno); ou podem ser **estatísticos** como eliminar dados correlacionadas (que podem ser redundantes) ou com baixa variância (que não carregam muita informação diferente para cada exemplo).

Os métodos **wrapper** podem ser divididos em **Backward elimination** e **Foward selection**, e se baseiam na ideia de selecionar as features através do treinamento de um modelo de aprendizado de máquina selecionado, escolhendo as features usadas a cada etapa de treinando e avaliando uma métrica de desemepenho. Esses métodos são automáticos mas podem gerar overfitting, são custosos em termos de tempo de computação e são dependendes do algoritmo de ML escolhido (features selecionadas para uma RandomForest podem não ser a mesma que para um KNN, por exemplo). 

Os métodos **embutidos** são detalhes internos do prório algoritmo de aprendizado de máquina. Por exemplo, a Regressão Ridge possui uma penalidade para variáveis correlacionadas na própria função de custo (loss function).

No artigo original dos dados aqui estudados (*Karthikeyan et al.*) há uma seleção por filtro de domínio, como as features sendo separadas por aquelas que consideram apenas informações 2D das moléculas e aquelas que consideram informações 3D das moléculas. Entretanto, os autores não fornceram as listas exatas de quais features são de cada um desses grupos.
Já no artigo de *Krstajic et al.*, os autores eliminaram 11 features com variação proxima de zero e 22 que eram combinações lineares das outras, restando 169 features. Novamente, contudo, eles não forneceram a lista das features eliminadas ou das restantes.

Vamos então adotar critérios próprios para proceder. Iremos utilizar uma combinação dos três métodos de seleção de features descritos acima. Iremos usar uma métrica de **Informação Mútua** para rankear as features que trazer o maior ganho de informação, pegando as **k-primeiras features**. Iremos utilizar o algoritmo de **Regressão Ridge** como nosso algoritmo de teste (pois teve bons resultados em nossa análise preliminar, consome pouco tempo e penaliza variáveis correlacionadas). Vejamos:

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression

In [ ]:
# Selecionando as k melhores features usando a medida de informação mútua
x = df_mp3.drop(columns=['Case','MTP', 'SMILE_Molecule']) # Estamos tirando a coluna de grupo e a coluna alvo 
y = df_mp3['MTP'] # Atributo alvo

# 2 Dividindo conjunto de treinamento e conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

In [ ]:
# Listas para armazenar os resultados
r2_ridge=[]
mad_mae_ridge=[]

start_time = time.time() # Marcar o tempo de início

for k in range(10,201,10):
    
    # 2 Selecionando as features
    selector = SelectKBest(score_func = mutual_info_regression, k=k)
    selector.fit(x_train, y_train)
    mask = selector.get_support()
    new_x_train = x_train[x.columns[mask]] # Estamos selecionando apenas as colunas definidas pelo seletor de features
    new_x_test = x_test[x.columns[mask]]   # Estamos selecionando apenas as colunas definidas pelo seletor de features
    
    # 3 Transformação de Escalonando
    scaler = MinMaxScaler()
    scaler.fit(new_x_train)
    x_train_scaled = scaler.transform(new_x_train)
    x_test_scaled = scaler.transform(new_x_test)
    
    # 4 Treinamento Ridge Regression
    rdg = Ridge()
    rdg.fit(x_train_scaled, y_train)

    # 5 Métricas Ridge Regression
    y_pred = rdg.predict(x_test_scaled)        
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)             
    r2_ridge.append(r2)
    mad_MTP = y_test.mad()  # Desvio absoluto médio de y_test
    mad_mae_ridge.append(mad_MTP/mae)

    #print(k)

print("Tempo de execução da seleção de features: {:.3f} segundos".format(time.time() - start_time))

Plotando um gráfico do desempenho do algoritmo de em função no número de features utilizadas, podemos ver que o desempenho é somente marginalmente melhor para mais de 100 features:

In [ ]:
k = [i for i in range(10,201,10)]

fig, ax1 = plt.subplots( figsize=(5,3))

# Plotando a curva de y1 no eixo esquerdo
ax1.plot(k, r2_ridge, 'b-')
ax1.set_xlabel('Quantidade de Features')
ax1.set_ylabel(r'$R^2$', color='b')
ax1.set_xlim([0,200])
ax1.set_ylim([0.2,0.6])
ax1.grid(True)
ax2 = ax1.twinx() # Criando o segundo eixo de valores y

# Plotando a curva de y2 no eixo direito
ax2.plot(k, mad_mae_ridge, 'r-')
ax2.set_ylabel('MAD:MAE', color='r')
ax2.set_ylim([1.0,1.6])

plt.show()

### 2 - Extra: Retornando ao quatro passo, Ajuste de hiperparâmetros do modelo

Podemos relizar um ajuste fino dos parâmetros de cada algoritmo utilizado. Vamos utilizar a técnica de GridSearch para variar os hiperparâmetros dos algoritmos de Ridge e Multiplayer Perceptron de maneira a encontrar uma solução subótima.

In [ ]:
from sklearn.model_selection import GridSearchCV

Vamos selecionar apenas as 100 primeiras features para prosseguir:

In [ ]:
# Selecionando as features
selector = SelectKBest(score_func = mutual_info_regression, k=100)
selector.fit(x_train, y_train)
mask = selector.get_support()
#x.columns[mask]

new_x_train = x_train[x.columns[mask]] # Estamos selecionando apenas as colunas mais importantes
new_x_test = x_test[x.columns[mask]] # Atributo alvo

In [ ]:
#new_x_train.info()

In [ ]:
# 3 Transformação de Escalonando
scaler = MinMaxScaler()
scaler.fit(new_x_train)
x_train_scaled = scaler.transform(new_x_train)
x_test_scaled = scaler.transform(new_x_test)

In [ ]:
# 4 Definindo os modelos
models = [
    Ridge(solver='lsqr'),
    MLPRegressor(random_state=42, solver='lbfgs'),
    #GradientBoostingRegressor(random_state=42)
]

# Definindo as grades de parâmetros para busca em grade para cada modelo
param_grids = [
    {'alpha': [0.01, 0.1, 1, 10, 100]}, # Hiperparametros do Ridge
    {'hidden_layer_sizes': [(26,12,), (30,17,), (17,10,), (100,50,30,)],
     'activation': ['relu', 'tanh']}, # Hiperparametros do MLP
    #{'n_estimators': [50, 100], 'max_depth': [2, 4]} # Hiperparametros do GB
]

resultados={} # Para armazenar os resultados

In [ ]:
# Iterando sobre cada modelo e sua respectiva grade de parâmetros
for model, param_grid in zip(models, param_grids):
    
    start_time = time.time()
    # Definindo o objeto GridSearchCV
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=5)
    
    # Treinando o modelo
    grid.fit(x_train_scaled, y_train)
    
    # Imprimindo os melhores parâmetros e score
    print('Modelo:', model.__class__.__name__)
    print('Melhores parâmetros:', grid.best_params_)
    print('Menor MAE treino:', -grid.best_score_)
    print("Tempo de otimização: {:.3f} segundos".format(time.time() - start_time))

    # Fazendo previsões no conjunto de teste
    y_pred = grid.predict(x_test_scaled)

    # Avaliando o desempenho do modelo no conjunto de teste
    mad_MTP = y_test.mad()  # Desvio absoluto médio de y_test
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)     
    
    # Salvando resultados
    rg_name = model.__class__.__name__
    # Salvando resultados
    scoring = {'MAE': mae,
               'R2' :r2,
               'MAD:MAE':(mad_MTP)/mae
         }
    resultados[rg_name]=scoring

In [ ]:
resultado_teste_regressao = pd.DataFrame(data=resultados)
resultado_teste_regressao.head()

In [ ]:
# Treino e teste com os melhores parâmetros
ridge = Ridge(solver='lsqr', alpha= 0.01)
ridge.fit(x_train_scaled, y_train)
y_pred_ridge = ridge.predict(x_test_scaled)

mlp = MLPRegressor(random_state=42, solver='lbfgs', activation = 'relu', hidden_layer_sizes=(30, 17,))
mlp.fit(x_train_scaled, y_train)
y_pred_mlp = mlp.predict(x_test_scaled)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10,4))

sns.scatterplot(y_test, y_pred_ridge, color='b', alpha=0.6,  ax = ax[0])
sns.lineplot(y_test, y_test, color='r', ax = ax[0])
ax[0].set_title('Regressão Ridge')
ax[0].set_ylabel(r'$y_{pred}$ (°C)'), ax[0].set_xlabel(r'$y_{test}$ (°C)')
ax[0].set_xlim([0,400]), ax[0].set_ylim([0,400])
ax[0].grid(True)

sns.scatterplot(y_test, y_pred_mlp, color='g', alpha=0.6,  ax = ax[1])
sns.lineplot(y_test, y_test, color='r', ax = ax[1])
ax[1].set_title('Regressão MLP')
ax[1].set_ylabel(r'$y_{pred}$ (°C)'), ax[1].set_xlabel(r'$y_{test}$ (°C)')
ax[1].set_xlim([0,400]), ax[1].set_ylim([0,400])
ax[1].grid(True)

fig.subplots_adjust(wspace=0.3, hspace=0.3)
plt.show()

--------------------------

## Exercícios propostos

1) Reproduzir os preditores de temperatura crítica de supercondutividade reportados em Stanev et al. 2018, https://www.nature.com/articles/s41524-018-0085-8.

2) Reproduzir o preditor de energia de bandgap de sólidos inorgânicos reportado em Zhuo et al. 2018, https://pubs.acs.org/doi/10.1021/acs.jpclett.8b00124. 